# Understand how to use pre-trained models for transfer learning.
Load a pre-trained model (VGG16) using TensorFlow/Keras or PyTorch.
Fine-tune the model on a custom dataset (flower classification).


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import tensorflow_datasets as tfds

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5

# Load dataset from TensorFlow Datasets
dataset, info = tfds.load("tf_flowers", with_info=True, as_supervised=True)
total_examples = info.splits['train'].num_examples
class_names = info.features['label'].names
num_classes = len(class_names)

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)  # VGG16 preprocessing
    label = tf.one_hot(label, num_classes)
    return image, label

# Apply preprocessing
data = dataset['train'].map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
data = data.shuffle(1000)

# Train-validation split (70-30)
train_size = int(0.7 * total_examples)
val_size = total_examples - train_size

train_ds = data.take(train_size).batch(BATCH_SIZE).repeat().prefetch(tf.data.AUTOTUNE)
val_ds = data.skip(train_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Calculate steps per epoch
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = val_size // BATCH_SIZE

# Build model with frozen VGG16 base
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps
)

# Plot training results
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

# Prediction visualization
def display_predictions(dataset, num_images=5):
    images = []
    labels = []
    for img, label in dataset.unbatch().take(num_images):
        images.append(img)
        labels.append(label)
    images = tf.stack(images)
    labels = tf.stack(labels)

    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)

    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        img = images[i].numpy()
        img = ((img - img.min()) / (img.max() - img.min()))  # Normalize to [0,1] for display
        plt.imshow(img)
        plt.axis('off')
        actual = class_names[true_classes[i]]
        predicted = class_names[pred_classes[i]]
        plt.title(f"Actual: {actual}\nPredicted: {predicted}", fontsize=10)
    plt.show()

# Call visualization on validation set
display_predictions(val_ds)


ModuleNotFoundError: No module named 'tensorflow_datasets'